# Access Token 생성
- 학습된 모델을 Huggingface hub에 올리기 위해서는 access token이 필요하다.
  
![huggingface_create_apikey.png](figures/huggingface_accesstoken.png)
![huggingface_create_apikey.png](figures/huggingface_accesstoken2.png)

- 1. 로그인 -> 2. Profile -> 3. Access Tokens 선택
- 생성할 때 `write` 권한을 선택한다.

In [ ]:
%pip install python-dotenv

  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# working directory / .env 생성

In [ ]:
from dotenv import load_dotenv
load_dotenv()
# working directory에서 환경변수 파일 (default: .env) 찾아서 그 파일에 설정된 값을 O/S의 환경변수로 등록

In [ ]:
# 환경변수 값을 읽기
import os
os.getenv("JAVA_HOME")

'/Library/Java/JavaVirtualMachines/jdk-21.jdk/Contents/Home'

In [ ]:
hf_api_key = os.getenv("HUGGINGFACE_API_KEY")
# os.environ["HUGGINGFACE_API_KEY"]

# Naver 영화댓글 분류

# Huggingface Dataset 패키지
- 허깅페이스 허브에 공유된 데이터셋을  다운로드해서 전처리 및 관리할 수있도록 돕는 라이브러리. 
- 많은 공개데이터셋을 동일한 인터페이스로 사용할 수있다.
- 설치
    - `pip install datasets`
- https://huggingface.co/datasets
- https://github.com/huggingface/datasets
      
## Huggingface Dataset loading
- datasets 로딩
    - `load_data('dataset name')`
        - huggingface datasets에 등록된 Dataset 이름 넣어 Loading한다.
          
![img](figures/huggingface_dataset.png)

In [9]:
# NSMC 데이터셋 로딩
import datasets
from datasets import load_dataset
nsmc = load_dataset("e9t/nsmc", trust_remote_code=True)
# trust_remote_code=True: 데이터셋 로딩 시 믿을 수 있는 파일이라고 인식
type(nsmc)

README.md:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

nsmc.py:   0%|          | 0.00/3.18k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

datasets.dataset_dict.DatasetDict

In [10]:
nsmc

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})

In [ ]:
# Dataset : 데이터셋
# DatasetDict : 여러 개의 데이터셋을 포함하는 딕셔너리. train, test, validation 등 여러 개의 데이터셋을 포함할 수 있음

In [ ]:
trainset = nsmc["train"]
testset = nsmc["test"]
trainset, testset
# 데이터셋이 표데이터일 경우 features는 컬럼명

(Dataset({
     features: ['id', 'document', 'label'],
     num_rows: 150000
 }),
 Dataset({
     features: ['id', 'document', 'label'],
     num_rows: 50000
 }))

In [14]:
# feature 조회
trainset["document"][:5]
trainset["label"][:5]
trainset["id"][:5]

['9976970', '3819312', '10265843', '9045019', '6483659']

In [15]:
# datasets.Dataset을 다른 형식으로 변환
# dataset 객체.to_xxx() -> 메서드를 다른 형식으로 변환
df = trainset.to_pandas()
df.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [16]:
# 다른 형식으로 저장된 data들을 datasets.Dataset으로 변환
# datasets.Dataset.from_pandas(df) -> pandas DataFrame을 datasets.Dataset으로 변환
d = datasets.Dataset.from_pandas(df)
d

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 150000
})

In [18]:
# Sampling - train : 10000개. test : 5000개
sample_trainset = trainset.shuffle().select(range(10000))
sample_testset = testset.shuffle().select(range(5000))

In [19]:
print(sample_trainset)
print(sample_testset)

Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 10000
})
Dataset({
    features: ['id', 'document', 'label'],
    num_rows: 5000
})


## 모델, 토크나이저 loading

- 모델 별 Model 클래스를 이용하거나 Auto class를 이용해 모델, 전처리기(tokenizer, ImageProcessor 등)을 로딩한다.
    - Huggingface에 저장된 model name을 입력해서 pretrained 모델을 loading 한다.
    - fine tuning 한 경우 모델 저장 디렉토리 경로를 넣어 pretrained 모델을 loading한다.
- AutoModel은 model name을 주면 그 모델이 학습한 base 모델에 맞는 객체를 생성해서 반환한다.
    - Auto Model은 task 별로 다양한 클래스들이 있다.
        - 클래스 이름 형식: AutoModelFor{Task형식}
        - ex) `AutoModelForObjectDetection`, `AutoModelForSequenceClassification`
    - https://huggingface.co/docs/transformers/model_doc/auto
    - 전처리기(tokenzier)는 사용하려는 모델이 사용한 전처리기를 사용해야 한다.

In [ ]:
# model_id = "beomi/kcbert-base" -> 문장의 특성을 추출하는 모델 (분류모델이 아님)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_id = "beomi/kcbert-base"                  # feature extractor
# 토크나이저, 분류 모델을 로딩 - tokenizer와 모델은 동일한 모델 ID를 사용해야 함
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=2)  # num_labels: 분류할 레이블의 개수(2개: 긍정/부정)
# model_id : kcbert-base -> pre-trained된 feature extractor 모델
# model : estimator는 학습 안된 layer로 구성

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
print(model)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## pytorch Dataset 생성
모델 입력으로 다음 4개 항목을 dictionary로 묶어서 제공하도록 구현한다.
1. input_ids: 입력 text 토큰을 id로 변환한 값
2. token_type_ids: 문자쌍 구분시 사용. 단일 문장: 0, 문자쌍-첫문장: 0, 두 번째 문장: 1
3. attention_mask: 실제 토큰값과 패딩구분값
4. labels: 정답 class index

1 ~ 3은 위의 train_encoding, test_encoding으로 만듬. labels은 train_data/test_data의 label 키 값 사용

In [ ]:
train_X = sample_trainset["document"]
train_y = sample_trainset["label"]

test_X = sample_testset["document"]
test_y = sample_testset["label"]

In [ ]:
# X(댓글)을 토큰화
train_encoding = tokenizer(train_X, return_tensors="pt", padding=True)   # train_X에서 가장 긴 문장 길이에 맞춰 padding
test_encoding = tokenizer(test_X, return_tensors="pt", padding=True)

In [38]:
train_encoding.keys()  # 토큰화된 결과의 키 확인
train_encoding["input_ids"].shape  # [10000: batch-문서수, 121: 토큰수]

torch.Size([10000, 121])

In [39]:
##########################################
# pytorch의 Dataset을 정의, 생성
##########################################
import torch
from torch.utils.data import Dataset
class NSMCDataset(Dataset):
    def __init__(self, comments, labels):
        """
        Args:
            comments(dict) : tokenizer로 토큰화한 input data들
            labels(list) : 정답 레이블들
        """
        self.comments = comments
        self.labels = labels

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, index):
        """
        index번째 데이터를 반환
        BERT 모델 입력 형식에 맞춰서 반환. (input_ids, token_type_ids, attention_mask) + labels
        Args:
            index(int) : 데이터의 인덱스
        Returns:
            dictionary - input_ids, token_type_ids, attention_mask, labels
                         입력데이터와 정답 label을 딕셔너리에 묶어서 반환
        """
        data = {key: val[index] for key, val in self.comments.items()}
        # idx = 3 이라면
        # data = {"input_ids":{4}, "token_type_ids":{40}, "attention_mask":{400}} 가 된다. -> 지정한 index 값만 저장
        data['labels'] = torch.tensor([self.labels[index]], dtype=torch.int64)
        return data

In [40]:
train_set = NSMCDataset(train_encoding, train_y)
test_set = NSMCDataset(test_encoding, test_y)

In [41]:
len(train_set), len(test_set)

(10000, 5000)

# 학습
- Transformers는 model 학습을 위해 TrainingArguments, Trainer 클래스를 제공한다.
- TrainingArguments는 Trainer를 위한 설정을 하는 클래스
- TrainingArguments, Trainer를 이용하면 training option, logging, gradient accumulation, mixed precision등을 쉽게 설정해 학습, 평가를 모두 진행할 수 있다.

In [ ]:
from transformers import Trainer, TrainingArguments
N_EPOCHS = 1
BATCH_SIZE = 64
train_args = TrainingArguments(
    output_dir="models/nsmc",                                             # train된 모델 저장
    num_train_epochs=N_EPOCHS,                                            # 학습 epoch 수
    per_device_train_batch_size=BATCH_SIZE,                               # 학습 배치 사이즈
    per_device_eval_batch_size=BATCH_SIZE,                                # 검증 배치 사이즈
    
    # 선택한 전략 단위로 검증 수행
    eval_strategy="steps",                                                 # 평가 전략 ["no", "step", "epoch"]의 선택지 존재
    save_strategy="steps",                                                # 모델 저장 전략
    save_total_limit=1,                                                   # 저장할 모델의 최대 개수
    load_best_model_at_end=True,                                          # 학습이 끝난 후 가장 좋은 모델을 저장 및 로드
    # True: eval_strategy, save_strategy가 같아야 함.

    metric_for_best_model="eval_loss",                                    # 가장 좋은 모델을 선택할 때 사용할 검증 기준 평가지표
    greater_is_better=False,                                              # eval_loss가 낮을수록 좋은 모델이므로 False로 설정

    report_to="none"                                                       # 로그 출력 설정 ["none", "tensorboard", "wandb"] -> default:wandb , 쓰지 않겠다 설정

    # huggingface-hub에 올리는 설정
    # push_to_hub=True,
    # hub_model_id="kcbert-nsmc-10000",                                     # 모델을 올릴 허깅페이스 모델 ID
    # hub_token=hf_api_key                                                  # 허깅페이스 API 키
)

In [36]:
# 평가 함수 정의 - evaluate 패키지 이용
# huggingface에서 제공하는 라이브러리. 다양한 평가함수들을 제공
%pip install evaluate scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [45]:
import evaluate

# 정확도 평가함수
acc_fn = evaluate.load("accuracy")

In [46]:
pred = torch.tensor([0, 1, 0, 1])
ref = torch.tensor([0, 1, 0, 0])
acc_fn.compute(predictions=pred, references=ref)

{'accuracy': 0.75}

In [ ]:
def compute_metrics(pred):
    """
    모델이 학습하는 도중 예측값을 받아서 평가 점수를 계산하는 함수
    Trainer에 의해 호출될 함수
    Args:
        pred(EvalPrediction) : 예측값과 정답 레이블을 포함하는 객체
    Returns:
        dictionary - key:평가지표 이름, value:평가지표 점수
    """
    labels = pred.label_ids  # 정답 레이블
    preds = pred.predictions.argmax(axis=-1)  # 모델의 출력값
    metrics1 = evaluate.load("accuracy")  # 정확도 평가 함수 로딩
    metrics2 = evaluate.load("f1")  # F1-score 평가 함수 로딩

    acc = metrics1.compute(predictions=preds, references=labels)  # 정확도 계산
    f1 = metrics2.compute(predictions=preds, references=labels, average="macro")  # 재현율 계산
    return {"accuracy": acc, "f1 score": f1}

In [54]:
# Trainer 객체 생성
trainer = Trainer(
    model=model,  # 학습할 모델
    args=train_args,  # 학습 인자
    train_dataset=train_set,  # 학습 데이터셋
    eval_dataset=test_set,  # 검증 데이터셋
    compute_metrics=compute_metrics  # loss 이외에 검증/평가 시 확인할 지표를 계산하는 함수
)

In [55]:
trainer.train()  # 모델 학습 시작

/Users/giwonjun/anaconda3/envs/dl/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss


RuntimeError: MPS backend out of memory (MPS allocated: 8.72 GB, other allocations: 335.42 MB, max allowed: 9.07 GB). Tried to allocate 90.75 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [56]:
trainer.evaluate()  # 검증 데이터셋에 대한 평가 수행

/Users/giwonjun/anaconda3/envs/dl/lib/python3.12/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


RuntimeError: MPS backend out of memory (MPS allocated: 8.79 GB, other allocations: 281.44 MB, max allowed: 9.07 GB). Tried to allocate 78.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [57]:
# 저장 - local directory
# tokenizer와 model을 같이 같은 경로에 저장
save_path = "models/nsmc"
tokenizer.save_pretrained(save_path)
model.save_pretrained(save_path)

## 모델 로드

In [58]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

save_path = "models/nsmc"
load_tokenizer = AutoTokenizer.from_pretrained(save_path)
load_model = AutoModelForSequenceClassification.from_pretrained(save_path)

# 추론

In [60]:
sentence = ["이걸 영화라고 만든 거냐?", "아무 기대 없이 봤는데 재미있네.", "내가 감독이어도 이것보다 재미있게 만들겠다.", "시간이 어떻게 가는 줄 모르고 봤다."]

In [61]:
from transformers import pipeline
pipe = pipeline(task="text-classification", model=load_model, tokenizer=load_tokenizer, device="cpu")
result = pipe(sentence)

Device set to use cpu


In [62]:
result

[{'label': 'LABEL_0', 'score': nan},
 {'label': 'LABEL_0', 'score': nan},
 {'label': 'LABEL_0', 'score': nan},
 {'label': 'LABEL_0', 'score': nan}]

In [64]:
### 학습한 모델, 토크나이저를 huggingface hub에 업로드 ###
# 1. TrainingArguments에 설정해서 학습 도중/끝나면 자동으로 업로드 되게 설정
# 2. model/tokenizer.push_to_hub("계정/모델id") 메서드를 이용해서 수동으로 업로드

# 로그인
from huggingface_hub import login
from dotenv import load_dotenv
# login(token=hf_api_key)  # 환경변수에 저장된 huggingface API key를 이용해서 로그인
# login()                  # Token에 Access Token을 입력해서 로그인
load_dotenv()  # .env 파일에서 환경변수 로딩

True

In [65]:
import os
hf_api_key = os.getenv("HUGGINGFACE_API_KEY")
login(hf_api_key)

In [66]:
model_id = "kcbert-nsmc-10000"
load_tokenizer.push_to_hub(model_id)  # 토크나이저를 huggingface hub에 업로드
load_model = model.push_to_hub(model_id)  # 모델을 huggingface hub에 업로드

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]